In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Benchmark

In [7]:
a = [1,2,3]
b = [1,2,3]

a == b

True

# Clone

In [88]:
dst = ViT()
dst.embedding.state_dict()

OrderedDict([('cls_token',
              tensor([[[ 0.5900, -1.7115, -0.3799, -1.1737, -1.3522,  1.2476, -0.1131,
                         0.5263, -0.0913, -0.8882, -0.1633,  1.1767,  0.5236, -0.8277,
                         0.7619,  1.1423, -0.0305,  0.1122,  0.0198,  0.6153,  1.8497,
                         0.8101,  1.3175,  1.2026, -0.6901, -0.6397, -0.4409,  0.4504,
                        -0.2047, -0.0272,  0.7718, -2.3402,  0.1700, -1.4384, -0.1587,
                         0.9391,  0.6130, -0.3367,  0.6688, -0.0990, -0.3199, -1.6032,
                         1.9512, -0.5551, -1.2153,  1.6378,  0.7767,  0.3395,  0.0610,
                         1.1436,  0.5334, -0.4963,  0.3517, -0.1968,  0.4588,  1.2061,
                        -0.3842, -0.0418,  0.8565,  0.4937, -1.9154, -0.6947,  0.0486,
                         0.7313, -0.8155, -0.2624, -0.1799,  2.5823, -0.0116, -0.2077,
                         0.1386, -0.2878,  0.4487, -0.2127,  0.9555, -0.4534,  1.2566,
                

In [38]:
import torch
from glasses.models import AutoModel, AutoConfig, EfficientNetLite
import timm
from transfer_weights import clone_model
from benchmark import benchmark
from glasses.models.classification.vit import ViT

src = timm.create_model('vit_base_patch16_384', pretrained='True')
dst = ViT.vit_base_patch16_384()

# dst = VisionTransformer()
dst = clone_model(src, dst, torch.randn((1, 3, 384, 384))).eval()

In [37]:
timm.list_models('vit*')

[autoreload of glasses.models.AutoConfig failed: Traceback (most recent call last):
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/zuppif/anaconda3/envs/dl/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 779, in exec_module
  File "<frozen importlib._bootstrap_external>", line 916, in get_code
  File "<frozen importlib._bootstrap_external>", line 846, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_

['vit_base_patch16_224',
 'vit_base_patch16_384',
 'vit_base_patch32_384',
 'vit_base_resnet26d_224',
 'vit_base_resnet50d_224',
 'vit_huge_patch16_224',
 'vit_huge_patch32_384',
 'vit_large_patch16_224',
 'vit_large_patch16_384',
 'vit_large_patch32_384',
 'vit_small_patch16_224',
 'vit_small_resnet26d_224',
 'vit_small_resnet50d_s3_224']

In [39]:
transform = AutoConfig.from_name('vit_base_patch16_384').transform

benchmark(src.cuda().eval(), transform, batch_size=128)

Using downloaded and verified file: ./imagenet_val_targets.pkl


0.9765625


(None, None, None)

In [40]:
dst.embedding.positions.data.copy_(src.pos_embed.data.squeeze(0))
dst.embedding.cls_token.data.copy_(src.cls_token.data)

transform = AutoConfig.from_name('vit_base_patch16_384').transform

benchmark(dst.cuda().eval(), transform, batch_size=128)

Using downloaded and verified file: ./imagenet_val_targets.pkl


0.9765625


(None, None, None)

In [65]:
from torchsummary import summary
from glasses.models.classification.vit import ViT

summary(VisionTransformer(), (3, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
        PatchEmbed-2             [-1, 196, 768]               0
           Dropout-3             [-1, 197, 768]               0
         LayerNorm-4             [-1, 197, 768]           1,536
            Linear-5            [-1, 197, 2304]       1,771,776
           Dropout-6         [-1, 12, 197, 197]               0
            Linear-7             [-1, 197, 768]         590,592
           Dropout-8             [-1, 197, 768]               0
MultiHeadAttentionTimm-9             [-1, 197, 768]               0
         Identity-10             [-1, 197, 768]               0
        LayerNorm-11             [-1, 197, 768]           1,536
           Linear-12            [-1, 197, 3072]       2,362,368
             GELU-13            [-1, 197, 3072]               0
          Dropout-14            [-1

(tensor(86415592), tensor(86415592), tensor(329.6493), tensor(737.6217))

In [8]:
summary(timm.create_model('vit_base_patch16_224', pretrained=True), (3, 224, 224), device='cpu')

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-vitjx/jx_vit_base_p16_224-80ecf9dd.pth" to /home/zuppif/.cache/torch/hub/checkpoints/jx_vit_base_p16_224-80ecf9dd.pth


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 768, 14, 14]         590,592
        PatchEmbed-2             [-1, 196, 768]               0
           Dropout-3             [-1, 197, 768]               0
         LayerNorm-4             [-1, 197, 768]           1,536
            Linear-5            [-1, 197, 2304]       1,771,776
           Dropout-6         [-1, 12, 197, 197]               0
            Linear-7             [-1, 197, 768]         590,592
           Dropout-8             [-1, 197, 768]               0
         Attention-9             [-1, 197, 768]               0
         Identity-10             [-1, 197, 768]               0
        LayerNorm-11             [-1, 197, 768]           1,536
           Linear-12            [-1, 197, 3072]       2,362,368
             GELU-13            [-1, 197, 3072]               0
          Dropout-14            [-1, 19

(tensor(86415592), tensor(86415592), tensor(329.6493), tensor(737.6217))

# Pretrained weights

In [ ]:
from pathlib import Path

root = Path('/home/zuppif/Documents/glasses-weights/')

models = map(lambda x: x.stem, root.glob('*.pth'))

for model in models:
    print(f"'{model}': BasicUrlHandler('https://github.com/FrancescoSaverioZuppichini/glasses-weights/blob/main/{model}.pth?raw=true'),")

## Traced

In [94]:
from glasses.utils.Tracker import Tracker
import torch

def trace(model, x):
    return Tracker(model)(x)

In [17]:
x = torch.randn((1, 3, 224, 224))

trace(ViT(), x)

[Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16)),
 Rearrange('b e (h) (w) -> b (h w) e'),
 LayerNorm((768,), eps=1e-05, elementwise_affine=True),
 Linear(in_features=768, out_features=2304, bias=True),
 Dropout(p=0.0, inplace=False),
 Linear(in_features=768, out_features=768, bias=True),
 Dropout(p=0.0, inplace=False),
 LayerNorm((768,), eps=1e-05, elementwise_affine=True),
 Linear(in_features=768, out_features=3072, bias=True),
 GELU(),
 Dropout(p=0.0, inplace=False),
 Linear(in_features=3072, out_features=768, bias=True),
 Dropout(p=0.0, inplace=False),
 LayerNorm((768,), eps=1e-05, elementwise_affine=True),
 Linear(in_features=768, out_features=2304, bias=True),
 Dropout(p=0.0, inplace=False),
 Linear(in_features=768, out_features=768, bias=True),
 Dropout(p=0.0, inplace=False),
 LayerNorm((768,), eps=1e-05, elementwise_affine=True),
 Linear(in_features=768, out_features=3072, bias=True),
 GELU(),
 Dropout(p=0.0, inplace=False),
 Linear(in_features=3072, out_features=768, bias

In [95]:
x = torch.randn((1, 3, 224, 224))

trace(timm.create_model('vit_base_patch16_224', pretrained=True), x).traced

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (

[]

In [72]:
trace(timm.create_model('vit_base_patch16_224', pretrained=True), x).parametrized

[Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16)),
 LayerNorm((768,), eps=1e-06, elementwise_affine=True),
 Linear(in_features=768, out_features=2304, bias=True),
 Linear(in_features=768, out_features=768, bias=True),
 LayerNorm((768,), eps=1e-06, elementwise_affine=True),
 Linear(in_features=768, out_features=3072, bias=True),
 Linear(in_features=3072, out_features=768, bias=True),
 LayerNorm((768,), eps=1e-06, elementwise_affine=True),
 Linear(in_features=768, out_features=2304, bias=True),
 Linear(in_features=768, out_features=768, bias=True),
 LayerNorm((768,), eps=1e-06, elementwise_affine=True),
 Linear(in_features=768, out_features=3072, bias=True),
 Linear(in_features=3072, out_features=768, bias=True),
 LayerNorm((768,), eps=1e-06, elementwise_affine=True),
 Linear(in_features=768, out_features=2304, bias=True),
 Linear(in_features=768, out_features=768, bias=True),
 LayerNorm((768,), eps=1e-06, elementwise_affine=True),
 Linear(in_features=768, out_features=3072, bias=Tr

In [20]:
ViT()

ViT(
  (0): PatchEmbedding(
    (projection): Sequential(
      (0): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (1): Rearrange('b e (h) (w) -> b (h w) e')
    )
  )
  (1): TransformerEncoder(
    (0): TransformerEncoderBlock(
      (0): ResidualAdd(
        (fn): Sequential(
          (0): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (1): MultiHeadAttention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (att_drop): Dropout(p=0.0, inplace=False)
            (projection): Sequential(
              (0): Linear(in_features=768, out_features=768, bias=True)
              (1): Dropout(p=0.0, inplace=False)
            )
          )
        )
      )
      (1): ResidualAdd(
        (fn): Sequential(
          (0): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (1): FeedForwardBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU()
            (2): Dropout(p

In [19]:
timm.create_model('vit_base_patch16_224', pretrained=True)

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (